Based on **Francesco Pierfederici: Distributed Computing with Python, Chapter 2**

# An asynchronous example

Things become interesting when we start **organizing coroutines into complex  graphs**. 

For instance, we might want to **count the concurrence of multiple words in the input file**.

The below code is an example for this:

In [1]:
def coroutine(fn):
    def wrapper(*args, **kwargs):
        c = fn(*args, **kwargs)
        next(c)
        return c
    return wrapper

def cat(f, case_insensitive, child):
    if case_insensitive:
        line_processor = lambda l: l.lower()
    else:
        line_processor = lambda l: l

    for line in f:
        child.send(line_processor(line))

@coroutine
def grep(substring, case_insensitive, child):
    if case_insensitive:
        substring = substring.lower()
    while True:
        text = (yield)
        child.send(text.count(substring))

@coroutine
def count(substring):
    n = 0
    try:
        while True:
            n += (yield)
    except GeneratorExit:
        print(substring, n)

@coroutine
def fanout(children):
    while True:
        data = (yield)
        for child in children: #Here we will send data to multiple corutines listed in the children variable
            child.send(data)

In [2]:
cat(f=open('../data/pg2600.txt'), case_insensitive=True, child=fanout(children=[grep(substring=p,case_insensitive=True,child=count(p)) for p in ['love','hate','hope']]))

hate 103
love 677
hope 158


### Other tools:
Python 3.4 introduced a **new library for asynchronous I/O** called **asyncio**.

Python 3.5 introduced **coroutine types** via **async def** and **await**.

We will discuss them later.